In [13]:
from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow as tf
import os
import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioModel import predict, train
from pydub import AudioSegment, generators
import IPython.display as ipd
from IPython.core.display import display, HTML
from audioInput import getLaughTracks, getNoise, processWavFile
from audioDisplay import predictAudio
import random
from audioUtils import downloadYtAndPrepareAudio, shell, readFolder
from audioTrain import trainAndSaveAndPredict, trainForNoise

def getGenerator(generator, seconds):    
    audio = AudioSegment.empty()
    for i in range(seconds):
        audio = audio + generator().to_audio_segment()
    return audio
        
def buildNoise(path, seconds):
    if os.path.isfile(path):    
        shell('rm %s' % path)
    SAMPLE_RATE = 44100
    
    def curriedSine():
        return generators.Sine(440)

    data = AudioSegment.empty()
    for i in range(seconds):
        generator = random.choice([curriedSine, generators.WhiteNoise])
        data = data + getGenerator(generator, 1)
    #data = data + getGenerator(curriedSine, 3) + getGenerator(generators.WhiteNoise, 3)        
    
    audio = AudioSegment(data.get_array_of_samples(), sample_width=2, channels=1, frame_rate=44100)
    audio.export(path, format='wav')
    #display(ipd.Audio(path))    
    
def getModel(path):
    files = readFolder('model/%s' % path)
    if len(files) > 0:
        return '%s/%s' % (path, files[0])
    return None



In [14]:
buildNoise('data/noise.wav', 10)
predictAudio('data/noise.wav', 'noise_100', ['sine', 'const', 'noise'], number_of_classes=3) 

loading ./model/noise_100/20-20/model
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/noise_100/20-20/model


array([0, 2, 1, 1, 2, 0, 0, 2, 2, 0])

In [3]:
predictAudio('data/noise.wav', getModel('noise_200'), number_of_classes = 3)